# POISSON - PRIMERA PRUEBA

usar un modelo de Regresión de Poisson, que es más adecuado para predicciones de conteo como crímenes, y asegurarnos de que las predicciones sean no negativas.

In [2]:
import pandas as pd
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/CristopherGR/crime_prediction/main/procesado_de_datos/datos_finales.csv')

# Convertir fecha a formato datetime
df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])

# Extraer el año de la fecha de detención
df['anio'] = df['fecha_detencion'].dt.year

# Agrupar datos por provincia y año
df_grouped = df.groupby(['provincia', 'anio']).size().reset_index(name='crimenes')

# Filtrar los datos para incluir solo los años de 2019 a 2023
df_grouped = df_grouped[df_grouped['anio'].between(2019, 2023)]

# Preparar los datos para el modelo
X = df_grouped[['provincia', 'anio']]
y = df_grouped['crimenes']

# Codificación de variables categóricas
X = pd.get_dummies(X, columns=['provincia'], drop_first=True)

# Ajustar un modelo de regresión de Poisson
model = PoissonRegressor(max_iter=1000)
model.fit(X, y)

# Predicciones en los datos de entrenamiento
y_pred = model.predict(X)

# Cálculo de métricas de desempeño
mape = (abs((y - y_pred) / y).mean()) * 100
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'METRICAS DE DESEMPEÑO PARA POISSON')
print(f'  >   MAPE: {mape}%')
print(f'  >   MAE: {mae}')
print(f'  >   R²: {r2}')

# Crear DataFrame para predicciones de 2024 y 2025
future_years = pd.DataFrame({
    'provincia': df_grouped['provincia'].unique().repeat(2),
    'anio': [2024, 2025] * len(df_grouped['provincia'].unique())
})

# Codificación de variables categóricas para años futuros
X_future = pd.get_dummies(future_years, columns=['provincia'], drop_first=True)
X_future = X_future.reindex(columns=X.columns, fill_value=0)  # Asegurarse de que las columnas coincidan

# Realizar predicciones
future_years['prediccion_crimenes'] = model.predict(X_future)

# Asegurarse de que las predicciones sean no negativas
future_years['prediccion_crimenes'] = future_years['prediccion_crimenes'].clip(lower=0)

# Reordenar columnas para el resultado final
resultados_finales = future_years[['provincia', 'anio', 'prediccion_crimenes']]

resultados_finales


METRICAS DE DESEMPEÑO PARA POISSON
  >   MAPE: 9.119955501651908%
  >   MAE: 277.1383441886693
  >   R²: 0.9869921284157344


c:\Users\crist\anaconda3\Lib\site-packages\sklearn\linear_model\_glm\glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


,provincia,anio,prediccion_crimenes
0,AZUAY,2024,2978.442257
1,AZUAY,2025,2978.169900
2,BOLIVAR,2024,854.252390
3,BOLIVAR,2025,854.174275
4,CARCHI,2024,1911.147137
5,CARCHI,2025,1910.972377
6,CAÑAR,2024,1396.194951
7,CAÑAR,2025,1396.067279
8,CHIMBORAZO,2024,2336.689030
9,CHIMBORAZO,2025,2336.475358


# POISSON - SEGUNDA PRUEBA

### Aumento de iteraciones y escalado

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Cargar el dataset
df = pd.read_csv('https://raw.githubusercontent.com/CristopherGR/crime_prediction/main/procesado_de_datos/datos_finales.csv')

# Preprocesamiento de los datos
df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])
df['anio'] = df['fecha_detencion'].dt.year
df_grouped = df.groupby(['provincia', 'anio']).size().reset_index(name='crimenes')

# Filtrar los datos para incluir solo los años de 2019 a 2023
df_grouped = df_grouped[df_grouped['anio'].between(2019, 2023)]

# Preparar los datos para el modelo
X = df_grouped[['provincia', 'anio']]
y = df_grouped['crimenes']

# Codificación de variables categóricas
X = pd.get_dummies(X, columns=['provincia'], drop_first=True)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos y mejorar la convergencia del algoritmo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustar un modelo de regresión de Poisson
model = PoissonRegressor(max_iter=2000)
model.fit(X_train_scaled, y_train)

# Predicciones en los datos de prueba
y_pred = model.predict(X_test_scaled)

# Cálculo de métricas de desempeño
mape = (abs((y_test - y_pred) / y_test).mean()) * 100
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'METRICAS DE DESEMPEÑO PARA POISSON')
print(f'   MAPE: {mape}%')
print(f'   MAE: {mae}')
print(f'   R²: {r2}')

# Crear DataFrame para predicciones de 2024 y 2025
future_years = pd.DataFrame({
    'provincia': df_grouped['provincia'].unique().repeat(2),
    'anio': [2024, 2025] * len(df_grouped['provincia'].unique())
})

# Codificación de variables categóricas para años futuros
X_future = pd.get_dummies(future_years, columns=['provincia'], drop_first=True)
X_future = X_future.reindex(columns=X.columns, fill_value=0)  # Asegurarse de que las columnas coincidan

# Escalar los datos futuros
X_future_scaled = scaler.transform(X_future)

# Realizar predicciones
future_years['prediccion_crimenes'] = model.predict(X_future_scaled)

# Asegurarse de que las predicciones sean no negativas
future_years['prediccion_crimenes'] = future_years['prediccion_crimenes'].clip(lower=0)

# Reordenar columnas para el resultado final
resultados_finales = future_years[['provincia', 'anio', 'prediccion_crimenes']]

resultados_finales

METRICAS DE DESEMPEÑO PARA POISSON
   MAPE: 10.073550510066386%
   MAE: 363.44009151249344
   R²: 0.985105487113427


,provincia,anio,prediccion_crimenes
0,AZUAY,2024,2750.181048
1,AZUAY,2025,2628.179331
2,BOLIVAR,2024,720.501159
3,BOLIVAR,2025,688.538761
4,CARCHI,2024,1690.903912
5,CARCHI,2025,1615.893148
6,CAÑAR,2024,1202.377014
7,CAÑAR,2025,1149.037958
8,CHIMBORAZO,2024,2031.278223
9,CHIMBORAZO,2025,1941.167999


In [3]:
resultados_finales.to_csv('predicciones.csv', index=False)